# 데이터 정제 : 이상치 데이터 처리

In [1]:
import numpy as np
import pandas as pd

In [5]:
# 데이터 로딩 및 개요 확인
# 원래는 기사 별 클릭 데이터지만 소스를 찾을 수 없어 기존에 있는 student.csv로 대체
# click_data = pd.read_csv('../data/click_sample_data.csv', encoding='cp949')
student_data = pd.read_csv('../data/student.csv')
student_data.head(10)

,이름,학과,성적
0,아이유,국문과,3.0
1,김연아,수학과,1.0
2,홍길동,전자과,3.5
3,김은숙,컴퓨터,2.7
4,홍의선,물리,4.0


In [21]:
# 실습을 위해 이상치 만들기
student_data.loc[1,'성적'] = 100 # loc는 index와 key(컬럼 이름)로 접근
student_data.iloc[3, 2] = -1 # iloc는 무조건 순서로
student_copy = student_data.copy()
student_copy

,이름,학과,성적
0,아이유,국문과,3.0
1,김연아,수학과,100.0
2,홍길동,전자과,3.5
3,김은숙,컴퓨터,-1.0
4,홍의선,물리,4.0


In [10]:
# 데이터 개요 파악
student_copy.info()

# 성적만 숫자로 이루어진 연속형 데이터로 이루어져 있고 나머지는 범주형 데이터로 이루어져 있음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   이름      5 non-null      object 
 1   학과      5 non-null      object 
 2   성적      5 non-null      float64
dtypes: float64(1), object(2)
memory usage: 248.0+ bytes


In [11]:
student_copy['성적'].describe()

count      5.000000
mean      21.900000
std       43.704119
min       -1.000000
25%        3.000000
50%        3.500000
75%        4.000000
max      100.000000
Name: 성적, dtype: float64

성적 최소는 -1, 최대는 100으로 되어 있다. 75% 수준에서 4인데 일부 데이터가 극단치가 존재하는 것으로 보인다

### 이상치 처리하기

#### Z-score

In [12]:
# 수식 Z = (x-mean) / std.dev
# Z = (해당 관측치 - 관측치 변수 평균) / (관측치 변수의 표준편차)

In [13]:
# Z-score 컬럼 생성
student_copy['z-score'] = (student_copy['성적']-student_copy['성적'].mean())/np.std(student_copy['성적'])
student_copy.head()

# 데이터의 개수가 5개 밖에 되지 않아서 100이 평균을 많이 높이는 바람에
# z-score 즉, 통계적 계산 벙법으로 이상치를 검출하기는 쉽지 않다.

,이름,학과,성적,z-score
0,아이유,국문과,3.0,-0.483498
1,김연아,수학과,100.0,1.997946
2,홍길동,전자과,3.5,-0.470707
3,김은숙,컴퓨터,-1.0,-0.585825
4,홍의선,물리,4.0,-0.457916


In [14]:
# 원본 데이터 내 z-score 확인
student_copy.describe()

# min : -0.58
# max : 1.99
# 일반적으로 z-score -3 또는 3을 넘어가는 데이터를 이상치라 판단하는데,
# 이 예제는 앞서 말했듯이 데이터 표본이 너무 적어서 정확히 판별하기 어렵다.
# 따라서, 인적 자원으로 이상치를 판단하고 처리하겠다.

,성적,z-score
count,5.000000,5.000000e+00
mean,21.900000,2.220446e-17
std,43.704119,1.118034e+00
min,-1.000000,-5.858253e-01
25%,3.000000,-4.834977e-01
50%,3.500000,-4.707068e-01
75%,4.000000,-4.579158e-01
max,100.000000,1.997946e+00


In [15]:
# 이상치 처리하기 [삭제]
# z-score 기반 이상치 제거 후 데이터 차원 확인
student_copy = student_copy[(student_copy['z-score']<1.5)&(student_copy['z-score'] > -1.5)]
student_copy.info() # 데이터 개요 확인

# 행이 5개에서 1개가 빠졌다. 즉 100의 성적을 가진 행이 빠졌다.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 0 to 4
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   이름       4 non-null      object 
 1   학과       4 non-null      object 
 2   성적       4 non-null      float64
 3   z-score  4 non-null      float64
dtypes: float64(2), object(2)
memory usage: 160.0+ bytes


In [16]:
# 삭제 후 데이터 요약 통계 확인
student_copy.describe()

,성적,z-score
count,4.000000,4.000000
mean,2.375000,-0.499486
std,2.286737,0.058499
min,-1.000000,-0.585825
25%,2.000000,-0.509080
50%,3.250000,-0.477102
75%,3.625000,-0.467509
max,4.000000,-0.457916


In [22]:
# Scipy 내 z-score 함수 제공
from scipy.stats import zscore
student_copy = student_data.copy()

student_copy['z-score'] = zscore(student_copy['성적'])
student_copy = student_copy[(student_copy['z-score']<1.5)&(student_copy['z-score']>-1.5)]

display(student_copy)
print(student_copy.shape)

,이름,학과,성적,z-score
0,아이유,국문과,3.0,-0.483498
2,홍길동,전자과,3.5,-0.470707
3,김은숙,컴퓨터,-1.0,-0.585825
4,홍의선,물리,4.0,-0.457916


(4, 4)


#### IQR(Interquartile Range)

In [34]:
# IQR 판단 기반 이상치 처리
student_copy = student_data.copy()
student_copy

,이름,학과,성적
0,아이유,국문과,3.0
1,김연아,수학과,100.0
2,홍길동,전자과,3.5
3,김은숙,컴퓨터,-1.0
4,홍의선,물리,4.0


In [26]:
# 1, 3 분위수 Q1, Q3 구하기
q1 = student_copy['성적'].quantile(0.25)
q3 = student_copy['성적'].quantile(0.75)
print('q1 : {}, q3 : {}'.format(q1, q3))

# IQR 구하기
iqr = q3 - q1
iqr

q1 : 3.0, q3 : 4.0


1.0

In [36]:
# IQR 기반 이상치 제거하기
student_copy = student_copy[(student_copy['성적']>(q1-1.5*iqr))&(student_copy['성적']<(q3-1.5*iqr))]
display(student_copy)

# 이 역시 데이터 표본 자체가 적어서 1.5라는 숫자가 정확하지 않다.
# 하지만, 표본이 커지게 되면 일반적으로 q1-1.5*iqr ~ q3 - 1.5*iqr을 정상 데이터로 규정한다


,이름,학과,성적


### 이상치 대체하기

In [44]:
student_copy = student_data.copy()
student_copy

,이름,학과,성적
0,아이유,국문과,3.0
1,김연아,수학과,100.0
2,홍길동,전자과,3.5
3,김은숙,컴퓨터,-1.0
4,홍의선,물리,4.0


In [45]:
# 대체할 기준 정의
max_score = 4.5

In [46]:
# 정의된 기준으로 대체 후 비교를 위해 컬럼 복사
student_copy['new_score'] = student_copy['성적']

# 이상치 대체
student_copy.loc[student_copy['new_score']>max_score, 'new_score'] = max_score
display(student_copy)

,이름,학과,성적,new_score
0,아이유,국문과,3.0,3.0
1,김연아,수학과,100.0,4.5
2,홍길동,전자과,3.5,3.5
3,김은숙,컴퓨터,-1.0,-1.0
4,홍의선,물리,4.0,4.0


In [47]:
student_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 0 to 4
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   이름       4 non-null      object 
 1   학과       4 non-null      object 
 2   성적       4 non-null      float64
 3   z-score  4 non-null      float64
dtypes: float64(2), object(2)
memory usage: 160.0+ bytes


* 데이터 이상치 제거 및 대체는 모델링 과정에서 결과 도출의 왜곡을 방지하기위해 수행하는 것이기 때문에, 수행 여부에 따라 전체 결과 퀄리티가 달라진다. 따라서 반드시 진행해줘야 한다.
* 일반적으로 이상치 대체 및 변경은 기존 도메인 지식 및 현업 담당자와의 협의를 통해 진행해야 한다.